Imports

In [1]:
import logging
import ssl
import urllib3

from elasticsearch import Elasticsearch, JSONSerializer
from elasticsearch.connection import create_ssl_context
from elasticsearch.helpers import parallel_bulk
from elasticsearch.helpers import scan
import numpy as np
import pandas as pd

urllib3.disable_warnings() # This is insecure
logger = logging.getLogger(__name__)

Elastic search connection procedures:

In [13]:
def get_elastic_client(server="local", write=False):
    if server == "local":
        if write:
            serializer = NpJSONSerializer()
        else:
            serializer = JSONSerializer()
        return Elasticsearch(host="localhost",
                             port=9200,
                             serializer=serializer)
    elif server in {"dev", "horizon", "prod"}:
        # All the other servers are remote hosts with similar configs
        if server == "prod":
            host = "daimler-elastic.vpc.bigml.com"
        elif server == "horizon":
            host = "daimler-elastic.horizon.bigml.com"
        else:
            host = "daimler-elastic.dev.bigml.com"

        if write:
            serializer = NpJSONSerializer()
        else:
            serializer = JSONSerializer()
        # Set up ssl context to disable cert verification
        ssl_context = create_ssl_context()
        ssl_context.check_hostname = False
        ssl_context.verify_mode = ssl.CERT_NONE
        return Elasticsearch(host=host,
                             port=443,
                             http_auth=("dev", "paroafCa"),
                             serializer=serializer,
                             ssl_context=ssl_context,
                             use_ssl=True,
                             request_timeout=50,
                             timeout=50,
                             max_retries=5, 
                             retry_on_timeout=True)
    else:
        logger.warning("unknown server '%s'", server)
        return None

Instanciate Elastic Search connection to dev:

In [14]:
ES = get_elastic_client("dev")
print(ES)

<Elasticsearch([{}])>


## 1) Query

In [57]:
get_missing_WOP_welds = {
  "size" : 10000,
  "query" : {
    "bool" : {
      "must" : [
        {
          "range" : {
            "timestamp" : {
              "from" : "2020-06-01",
              "to" : "2020-11-01",
              "include_lower" : True,
              "include_upper" : False,
              "boost" : 1.0
            }
          }
        },
        {
          "term" : {
            "WeldTimeActual" : {
              "value" : 0,
              "boost" : 1.0
            }
          }
        }
      ],
      "adjust_pure_negative" : True,
      "boost" : 1.0
    }
  },
  "_source" : {
    "includes" : [
      "fingerprint",
      "StudID",
      "CarbodyID",
      "FaultCode",
      "LMPenetrationActual",
      "LMLiftHeightActual",
      "WeldCurrentActualPositive",
      "WeldVoltageActual",
      "WeldEnergyActual",
      "WIP"
    ],
    "excludes" : [ ]
  },
  "sort" : [
    {
      "_doc" : {
        "order" : "asc"
      }
    }
  ]
}

In [54]:
REQUEST_TIMEOUT = 100.  # Timeout for the elastic queries

In [58]:
elastic_response = ES.search(index="ml_toolbox_raw_data",
                           body=get_missing_WOP_welds,
                           request_timeout=REQUEST_TIMEOUT)

In [59]:
print(elastic_response)

{'took': 91445, 'timed_out': False, '_shards': {'total': 24, 'successful': 24, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 5553, 'relation': 'eq'}, 'max_score': None, 'hits': [{'_index': 'raw-te1500ac-2020.08', '_type': '_doc', '_id': 'b90e83c90ea9c27472771f3732540fe4ce5c690f', '_score': None, '_source': {'FaultCode': '17035', 'WeldEnergyActual': 0, 'StudID': '6200114', 'CarbodyID': '99983405', 'LMPenetrationActual': '', 'fingerprint': 'b90e83c90ea9c27472771f3732540fe4ce5c690f', 'LMLiftHeightActual': '', 'WeldCurrentActualPositive': 0, 'WeldVoltageActual': 0, 'WIP': False}, 'sort': [208]}, {'_index': 'raw-te1500ac-2020.08', '_type': '_doc', '_id': '1ffe0b41df6f91d4cf34e10d1d66f76939a3facc', '_score': None, '_source': {'FaultCode': '13009', 'WeldEnergyActual': 0, 'StudID': '6100236', 'CarbodyID': '99983298', 'LMPenetrationActual': '', 'fingerprint': '1ffe0b41df6f91d4cf34e10d1d66f76939a3facc', 'LMLiftHeightActual': '', 'WeldCurrentActualPositive': 0, 'WeldVoltageActual': 0, '

In [63]:
weld_stats_df = pd.DataFrame()

for bucket in elastic_response["hits"]["hits"]:
    # gather current bucket key information
    fingerprint = bucket["_id"]
    studid = bucket["_source"]["StudID"]
    carbody = bucket["_source"]["CarbodyID"]
    faultcode = bucket["_source"]["FaultCode"]
    penetration = bucket["_source"]["LMPenetrationActual"]
    weldEnergyActual = bucket["_source"]["WeldEnergyActual"]
    liftHeight = bucket["_source"]["LMLiftHeightActual"]
    currentPositive = bucket["_source"]["WeldCurrentActualPositive"]
    weldVoltageActual = bucket["_source"]["WeldVoltageActual"]
    wip = bucket["_source"]["WIP"]
    
    
    current_data_dict = {"Fingerprint" : [fingerprint], 
                         "StudId" : [studid], 
                         "CarbodyId" : [carbody], 
                         "FaultCode" : [faultcode], 
                         "LMPenetrationActual" : [penetration], 
                         "WeldEnergyActual" : [weldEnergyActual], 
                         "LMLiftHeightActual" : [liftHeight], 
                         "WeldCurrentActualPositive" : [currentPositive], 
                         "weldVoltageActual" : [weldVoltageActual],
                         "WIP" : [wip]}
    
    # create dataframe with current feature set results
    cur_bucket_df = pd.DataFrame(current_data_dict, columns = ['Fingerprint',
                                                               'StudId',
                                                               'CarbodyId',
                                                               'FaultCode',
                                                               'LMPenetrationActual',
                                                               'WeldEnergyActual',
                                                               'LMLiftHeightActual',
                                                               'WeldCurrentActualPositive',
                                                               'weldVoltageActual',
                                                               'WIP'])
    
    weld_stats_df = weld_stats_df.append(cur_bucket_df, ignore_index=True)

In [64]:
print(weld_stats_df.shape)

(5553, 10)


In [65]:
weld_stats_df.head()

,Fingerprint,StudId,CarbodyId,FaultCode,LMPenetrationActual,WeldEnergyActual,LMLiftHeightActual,WeldCurrentActualPositive,weldVoltageActual,WIP
0,b90e83c90ea9c27472771f3732540fe4ce5c690f,6200114,99983405,17035,,0,,0,0.0,False
1,1ffe0b41df6f91d4cf34e10d1d66f76939a3facc,6100236,99983298,13009,,0,,0,0.0,False
2,3c7c821cb988a55a5c77ddf394fa62ad5acd18d9,6100575,85067874,13011,,0,,0,0.0,False
3,2547f342cc8fe5f510a954e7b10bc7c17fc2be29,620406,26793257,13008,-0.28,1,0.6,0,0.0,False
4,42240b37ef1a83add0fbb06a611ebb7f3838cfc8,6100638,85086817,17051,,0,,0,0.0,False


In [66]:
weld_stats_df.to_csv('/Users/guillem/Data/Customers/Daimler/anomalies-analysis/empty_repairs_WIP.csv', index = False, header=True)

# September and October Missings (for repeats analysis)

In [67]:
get_missing_welds_SO = {
  "size" : 10000,
  "query" : {
    "bool" : {
      "must" : [
        {
          "range" : {
            "timestamp" : {
              "from" : "2020-09-01",
              "to" : "2020-11-01",
              "include_lower" : True,
              "include_upper" : False,
              "boost" : 1.0
            }
          }
        },
        {
          "term" : {
            "WeldTimeActual" : {
              "value" : 0,
              "boost" : 1.0
            }
          }
        }
      ],
      "adjust_pure_negative" : True,
      "boost" : 1.0
    }
  },
  "_source" : {
    "includes" : [
      "fingerprint",
      "StudID",
      "CarbodyID",
      "FaultCode",
      "LMPenetrationActual",
      "LMLiftHeightActual",
      "WeldCurrentActualPositive",
      "WeldVoltageActual",
      "WeldEnergyActual",
      "WIP"
    ],
    "excludes" : [ ]
  },
  "sort" : [
    {
      "_doc" : {
        "order" : "asc"
      }
    }
  ]
}

In [68]:
missing_welds_OS_df = pd.DataFrame()

num_batch = 1
welds = []

for current_weld in scan(ES, index="ml_toolbox_raw_data", 
                                  query=get_missing_welds_SO, 
                                  scroll='15m',
                                  raise_on_error=True,
                                  size=5000,
                                  request_timeout=REQUEST_TIMEOUT):

    if num_batch % 10000 == 0:
      print("Current batch %s" % num_batch)
    
    num_batch = num_batch + 1

    # gather current bucket key information
    welds.append(current_weld["_source"])


## build dataframe from welds list
print("Loop finished, building dataframe ")
print(len(welds))

missing_welds_OS_df = pd.DataFrame.from_records(welds)

Loop finished, building dataframe 
397


In [71]:
print(missing_welds_OS_df.shape)

(397, 10)


In [72]:
weld_stats_df.to_csv('/Users/guillem/Data/Customers/Daimler/anomalies-analysis/missing_welds_all_SO.csv', index = False, header=True)